In [1]:
import math
import fluids
from fluids.units import *


Given:  S.A.E. 10W Oil at 60 degF discharges from a tank with 22-feet average head to atmosphere through:
        200 feet - 3" Schedule 40 pipe
        6 - 3" standard 90 deg threaded elbows
        1 - 3" flanged ball valve having a 2-3/8" diameter seat, 16 deg conical inlet, and 30 deg conical outlet end.  Sharp-edged entrance is flush with the inside of the tank.
        
Find: Velocity of flow in the pipe and rate of discharge in gallons per minute.

In [2]:
T = (60*u.degF).to(u.degK)

Assume atmospheric pressures

In [3]:
P = (1*u.atm).to(u.Pa)

PubChem does not have SAE data.  Need to read these off Crane Appendixes

In [4]:
rho = 54.6*u.lb/u.feet**3 #Crane A-8
mu = 75*u.cP #Crane A-3

# Get the friction coefficient

In [5]:
d1 = (2+3/8)*u.inch
d2 = nearest_pipe(NPS=3*u.inch, schedule='40')[1].to(u.inch)
beta = d1/d2

#For oil, first assumption of laminar flow is appropriate
Re =1.5*10**3

e = roughness_Farshad('Carbon steel, bare', D=d2.to(u.meter))
f = friction_factor(Re=Re, eD=e/d2.to(u.meter))

f_check = 64/Re  #laminar flow friction formula

f, f_check  #Turns out the roughness_Fashard uses the liminary friction formula

(<Quantity(0.042666666666666665, 'dimensionless')>, 0.042666666666666665)

# Sum the Ks

In [6]:
K_inlet = 0.5 #Crane A-30
K_exit = 1.0 #Crane A-30



sin_theta_div2_inlet = math.sin(8*math.pi/180)
sin_theta_div2_outlet = math.sin(15*math.pi/180)

ft = 0.017 #To use the formulas for K2 from Crane 410, K1 is calculated using friciton at complete turbulence
            #need to get the ft from A-27.  The difference in using f as calculated for the remainder
            #is not large.
K1 = 3*ft
#Since inlet and outlet angles differ, Formula 5 from Crane A-29 needs to be modified
K2 = (K1 + 0.80*sin_theta_div2_inlet*(1-beta) + 2.6*sin_theta_div2_outlet*(1-beta**2)**2)/beta**4
K_valve = K2

#Elbows
n_elbows = 6
K_elbows = (30*f) * n_elbows

#Pipe
D = d2
L=200*u.feet
K_pipe= f*L/D.to(u.feet)

K_tot = K_inlet+K_exit+K_valve+K_elbows+K_pipe

print("Total resistance coefficint is {}.".format(K_tot))

Total resistance coefficint is 43.07247074284953 dimensionless.


In [7]:
#velocity of flow in the pipe
g=32.2*u.feet/u.sec**2
hl = 22*u.feet
v = math.sqrt((2*g*hl/K_tot).magnitude)*u.feet/u.sec  #math module doesn't handle units

#flow in pipe
Q =(2.448*v*d2**2).magnitude*u.gallons/u.minute

print(" Pipe flow velocity is: {}\n Flow rate is: {}".format(v,Q))

 Pipe flow velocity is: 5.735276783902027 foot / second
 Flow rate is: 132.12842244229753 gallon / minute


In [8]:
#Calculate the RE number to make sure that we have realistic fricion factor
V = v.to(u.meter/u.sec)
Re_check = Reynolds(D=d2.to(u.meter), rho=rho.to(u.kg/u.meter**3), V=v, mu=mu.to(u.Pa*u.sec))
print("Reynolds number check is: {} - {} =  {}.".format(Re, Re_check, Re-Re_check))

Reynolds number check is: 1500.0 - 1588.4389219670084 dimensionless =  -88.43892196700835 dimensionless.


Interating the initial guess of Re verse the check R will give the soluton to be about Re=1620.  Our initial guess of laminar flow is correct.  The acceptable answer is ~6 ft/sec and ~130 gallons/min.  Our approach is to change the Re values to get the v to converge.

# Solution as Provided by Crane 410

In [9]:
K_valve = 0.59 #from problem with water flowing through the same configuration
K_elbow = 3.06 #from same as above
K_inlet = 0.5 #Crane A-30
K_exit = 1 #Crane A-30
rho = 54.4
mu = 75
hl=22

#Assume velocity
v=5

Re = (124*d2*v*rho/mu).magnitude

f = 64/Re #for pipe


D = d2.to(u.feet)


L = 200*u.feet

K_pipe = f*L/D

Ktot = K_valve + K_elbow + K_inlet + K_exit + K_pipe

v = math.sqrt((2*g*hl/Ktot).magnitude)

Q = 2.448*v*(d2.magnitude)**2

v, Q

(5.846890090985146, 134.69975260544737)

Need to guess one of {f,Re} or {v,Q} to start the solution.  Then iterate until v or Q converges.


The calculation from the fluids module vs what Crane 410 gives is about the same.